In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Layer, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

df = pd.read_csv('data/Ratings.csv')
users = df['userid']
items =df['itemid']
num_users = users.nunique()
num_items = items.nunique()
train_ratings = np.zeros((num_users, num_items))
train_ratings_df = pd.read_csv('data/train_ratings.csv')
for index, row in train_ratings_df.iterrows():
    user_id = row['userid']-1
    item_id = row['itemid']-1
    rating_value = row['rating']
    train_ratings[user_id,item_id]=rating_value

class AttentionLayer(Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)
    
    def build(self, input_shape):
        self.attention_weights = self.add_weight(name='attention_weights',
                                                 shape=(input_shape[-1], 1),
                                                 initializer='glorot_uniform',
                                                 trainable=True)
        super(AttentionLayer, self).build(input_shape)
    
    def call(self, inputs):
        attn_scores = tf.nn.tanh(tf.matmul(inputs, self.attention_weights))
        attn_scores = tf.nn.softmax(attn_scores, axis=1)
        attn_output = tf.reduce_sum(inputs * attn_scores, axis=1)
        return attn_output

hidden_dim = 30
learning_rate = 0.001
num_epochs = 20
batch_size = 32


# Model definition
input_layer = Input(shape=(num_items,1))
encoder = Dense(hidden_dim, activation='relu')(input_layer)
attention = AttentionLayer()(encoder)
decoder = Dense(num_items, activation='sigmoid')(attention)

autoencoder = Model(inputs=input_layer, outputs=decoder)
autoencoder.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse')

# Training
autoencoder.fit(train_ratings, train_ratings, epochs=num_epochs, batch_size=batch_size, shuffle=True)

# Extracting user and item features
encoder_model = Model(inputs=input_layer, outputs=attention)
user_ratings_features = encoder_model.predict(train_ratings)
item_ratings_features = autoencoder.layers[-1].get_weights()[0]

item_ratings_features=item_ratings_features.reshape(num_items,hidden_dim)
print("User Features Shape:", user_ratings_features.shape)
print("Item Features Shape:", item_ratings_features.shape)